In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import torch

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-06-01 19:50:41.187787: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
df = pd.read_csv('df_merged_rouge.csv', index_col='Unnamed: 0').reset_index(drop=True)

In [6]:
df

,title,generated_text,generated,model,generated_text_without_title,generated_text_clean,clean_text,text,rouge1_f1,rouge1_precision,rouge1_recall,rouge2_f1,rouge2_precision,rouge2_recall,rougeL_f1,rougeL_precision,rougeL_recall
0,В Испании арестован польский оружейный барон,**В Испании арестован польский оружейный барон...,1,YandexGPT,****\n\nПравоохранительные органы Испании арес...,правоохранительные органы испании арестовали г...,испанская полиция сообщила об аресте гражданин...,Испанская полиция сообщила об аресте гражданин...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
1,Пожар в жилом доме в Солнцево потушен,**Пожар в жилом доме в Солнцево потушен**\n\nС...,1,YandexGPT,"\n\nСегодня в жилом доме, расположенном в моск...",сегодня в жилом доме расположенном в московско...,пожар возникший в квартире жилого дома располо...,"Пожар, возникший в квартире жилого дома, распо...",0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
2,В Генпрокуратуре заявили о наличии обманутых д...,**В Генпрокуратуре заявили о наличии обманутых...,1,YandexGPT,\n\nГенеральная прокуратура России провела про...,генеральная прокуратура россии провела проверк...,проблемы с обманутыми дольщиками существуют в ...,Проблемы с обманутыми дольщиками существуют в ...,0.200000,1.000000,0.111111,0.0,0.0,0.0,0.200000,1.000000,0.111111
3,"Фонд ""Федерация"" отсудил у ""Коммерсанта"" три т...",**Фонд «Федерация» отсудил у «Коммерсанта» три...,1,YandexGPT,\n\nБлаготворительный фонд «Федерация» выиграл...,благотворительный фонд федерация выиграл дело ...,иск фонда федерация к газете коммерсантъ по по...,"Иск фонда ""Федерация"" к газете ""Коммерсантъ"" п...",0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
4,Российские банкиры считают невозможным снижени...,**Российские банкиры считают невозможным сниже...,1,YandexGPT,\n\nВ условиях экономической нестабильности и ...,в условиях экономической нестабильности и рост...,снижение ставок по ипотечным кредитам невозмож...,Снижение ставок по ипотечным кредитам невозмож...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24370,В Арктике нашли тропические леса,Новостная статья: В Арктике нашли тропические ...,1,RuGPT3_small pre-trained,", которые могут быть долгожителями. Ученые пр...",которые могут быть долгожителями ученые проана...,британские ученые в ходе экспедиции на норвежс...,Британские ученые в ходе экспедиции на норвежс...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
24371,Доллар упал ниже 70 рублей,Новостная статья: Доллар упал ниже 70 рублей в...,1,RuGPT3_small pre-trained,впервые с начала года. Об этом свидетельству...,впервые с начала года об этом свидетельствуют ...,курс доллара на московской бирже вечером в сре...,Курс доллара на Московской бирже вечером в сре...,0.103448,0.078947,0.150000,0.0,0.0,0.0,0.068966,0.052632,0.100000
24372,Подсчитаны доходы Роналду от Instagram,Новостная статья: Подсчитаны доходы Роналду от...,1,RuGPT3_small pre-trained,-аккаунта Роналдо. Роналдиньо является самым ...,аккаунта роналдо роналдиньо является самым выс...,нападающий мадридского реала криштиану роналду...,Нападающий мадридского «Реала» Криштиану Ронал...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
24373,Ученые нашли у кофе способность продлевать жизнь,Новостная статья: Ученые нашли у кофе способно...,1,RuGPT3_small pre-trained,"бактерий, которые производят кофеин. Исследо...",бактерий которые производят кофеин исследовани...,дветри чашки кофе в день снижают риск смерти о...,Две-три чашки кофе в день снижают риск смерти ...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000


In [7]:

if torch.cuda.is_available():
    device = torch.device('cuda')
    print("CUDA is available. GPU will be used.")
else:
    device = torch.device('cpu')
    print("CUDA is not available. Using CPU.")


model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2', device=device)

def batch_embed_texts(texts, batch_size=64):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i + batch_size]
        batch_embeddings = model.encode(batch_texts, convert_to_tensor=True, show_progress_bar=False, device=device)
        embeddings.append(batch_embeddings)
    return torch.cat(embeddings)

generated_embeddings = batch_embed_texts(df['generated_text_clean'].tolist())
clean_embeddings = batch_embed_texts(df['clean_text'].tolist())

generated_embeddings = generated_embeddings.cpu()
clean_embeddings = clean_embeddings.cpu()

def calculate_sts(generated_emb, reference_emb):
    similarity = util.pytorch_cos_sim(generated_emb, reference_emb)
    return similarity.item()

df['sts_score'] = [
    calculate_sts(generated_embeddings[i], clean_embeddings[i])
    for i in range(len(df))
]

del generated_embeddings
del clean_embeddings

print(df[['generated_text_clean', 'clean_text', 'sts_score']])


CUDA is available. GPU will be used.


/home/jupyter/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
100%|██████████| 381/381 [00:26<00:00, 14.30it/s]


                                    generated_text_clean  ... sts_score
0      правоохранительные органы испании арестовали г...  ...  0.887151
1      сегодня в жилом доме расположенном в московско...  ...  0.899150
2      генеральная прокуратура россии провела проверк...  ...  0.783440
3      благотворительный фонд федерация выиграл дело ...  ...  0.719282
4      в условиях экономической нестабильности и рост...  ...  0.815478
...                                                  ...  ...       ...
24370  которые могут быть долгожителями ученые проана...  ...  0.570741
24371  впервые с начала года об этом свидетельствуют ...  ...  0.706034
24372  аккаунта роналдо роналдиньо является самым выс...  ...  0.675326
24373  бактерий которые производят кофеин исследовани...  ...  0.587809
24374  оригинальное название trinity blade the night ...  ...  0.525621

[24375 rows x 3 columns]


In [8]:
df.to_csv('merged_df_similarity_metrics.csv')

In [9]:
df.groupby(by = ['model'])['sts_score'].agg(['min', 'max', 'mean', 'median'])

,min,max,mean,median
model,,,,
GigaChat,0.276286,0.949357,0.743014,0.756152
RuGPT3_small,-0.100604,0.926263,0.514622,0.533179
RuGPT3_small pre-trained,-0.052333,0.905642,0.535352,0.563443
YandexGPT,-0.193830,0.953546,0.753172,0.769002
YandexGPT 3 lite,0.278011,0.939900,0.729774,0.744395
YandexGPT lite,0.174363,0.956613,0.755190,0.771488


In [23]:
import shutil

total, used, free = shutil.disk_usage("/")

print("Total: %d GiB" % (total // (2**30)))
print("Used: %d GiB" % (used // (2**30)))
print("Free: %d GiB" % (free // (2**30)))

Total: 98 GiB
Used: 70 GiB
Free: 23 GiB


In [25]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = 'ai-forever/rugpt3medium_based_on_gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

model.eval()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

tqdm.pandas()

def calculate_perplexity(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs['input_ids'])
        loss = outputs.loss
    perplexity = torch.exp(loss).item()
    return perplexity

df['perplexity'] = df['generated_text_without_title'].progress_apply(calculate_perplexity)

print(df[['generated_text_without_title', 'perplexity']])

df.to_csv('/home/jupyter/datasphere/project/df_with_sts_perplexity.csv', index=False)


/home/jupyter/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
100%|██████████| 24375/24375 [15:23<00:00, 26.40it/s]

                            generated_text_without_title  perplexity
0      ****\n\nПравоохранительные органы Испании арес...    5.974008
1      \n\nСегодня в жилом доме, расположенном в моск...    5.574790
2      \n\nГенеральная прокуратура России провела про...    6.180483
3      \n\nБлаготворительный фонд «Федерация» выиграл...    9.139601
4      \n\nВ условиях экономической нестабильности и ...    7.054786
...                                                  ...         ...
24370   , которые могут быть долгожителями. Ученые пр...   18.511623
24371    впервые с начала года. Об этом свидетельству...   13.230332
24372   -аккаунта Роналдо. Роналдиньо является самым ...   21.669037
24373    бактерий, которые производят кофеин. Исследо...   19.451475
24374    (оригинальное название «Trinity Blade: The N...   17.617302

[24375 rows x 2 columns]


In [28]:
df['perplexity_original_text'].agg(['min', 'max', 'mean', 'median'])

min         3.042852
max       340.760223
mean       12.926409
median     12.158146
Name: perplexity_original_text, dtype: float64

In [27]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

#model_name = 'ai-forever/rugpt3medium_based_on_gpt2'
#model = GPT2LMHeadModel.from_pretrained(model_name)
#tokenizer = GPT2Tokenizer.from_pretrained(model_name)

#model.eval()

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model = model.to(device)

#tqdm.pandas()

def calculate_perplexity(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs['input_ids'])
        loss = outputs.loss
    perplexity = torch.exp(loss).item()
    return perplexity

df['perplexity_original_text'] = df['text'].progress_apply(calculate_perplexity)

print(df[['text', 'perplexity_original_text']])

df.to_csv('/home/jupyter/datasphere/project/df_with_perplexity.csv', index=False)


100%|██████████| 24375/24375 [14:58<00:00, 27.12it/s]


                                                    text  perplexity_original_text
0      Испанская полиция сообщила об аресте гражданин...                 11.997695
1      Пожар, возникший в квартире жилого дома, распо...                  6.068287
2      Проблемы с обманутыми дольщиками существуют в ...                  9.458086
3      Иск фонда "Федерация" к газете "Коммерсантъ" п...                 17.563833
4      Снижение ставок по ипотечным кредитам невозмож...                  8.867599
...                                                  ...                       ...
24370  Британские ученые в ходе экспедиции на норвежс...                 10.586896
24371  Курс доллара на Московской бирже вечером в сре...                  6.446386
24372  Нападающий мадридского «Реала» Криштиану Ронал...                  9.384760
24373  Две-три чашки кофе в день снижают риск смерти ...                 15.520304
24374  Киностудия Marvel представила трейлер фантасти...                 12.443316

[24

In [66]:
import torch
   
torch.cuda.empty_cache()
   
import gc
gc.collect()

24

In [2]:
df = pd.read_csv('df_with_perplexity.csv').drop_duplicates(subset=['title', 'model']).reset_index(drop=True)
df = df.drop(df[df['perplexity'] > 150].index)
df = df.drop(df[~ df.model.isin(['GigaChat','RuGPT3_small','YandexGPT lite', 'YandexGPT 3 lite', 'YandexGPT', 'RuGPT3_small pre-trained'])].index)
df

,title,generated_text,generated,model,generated_text_without_title,generated_text_clean,clean_text,text,rouge1_f1,rouge1_precision,rouge1_recall,rouge2_f1,rouge2_precision,rouge2_recall,rougeL_f1,rougeL_precision,rougeL_recall,sts_score,perplexity,perplexity_original_text,generated_sentiment
0,В Испании арестован польский оружейный барон,**В Испании арестован польский оружейный барон...,1,YandexGPT,****\n\nПравоохранительные органы Испании арес...,правоохранительные органы испании арестовали г...,испанская полиция сообщила об аресте гражданин...,Испанская полиция сообщила об аресте гражданин...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.887151,5.974008,11.997695,NEUTRAL
1,Пожар в жилом доме в Солнцево потушен,**Пожар в жилом доме в Солнцево потушен**\n\nС...,1,YandexGPT,"\n\nСегодня в жилом доме, расположенном в моск...",сегодня в жилом доме расположенном в московско...,пожар возникший в квартире жилого дома располо...,"Пожар, возникший в квартире жилого дома, распо...",0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.899150,5.574790,6.068287,POSITIVE
2,В Генпрокуратуре заявили о наличии обманутых д...,**В Генпрокуратуре заявили о наличии обманутых...,1,YandexGPT,\n\nГенеральная прокуратура России провела про...,генеральная прокуратура россии провела проверк...,проблемы с обманутыми дольщиками существуют в ...,Проблемы с обманутыми дольщиками существуют в ...,0.200000,1.000000,0.111111,0.0,0.0,0.0,0.200000,1.0,0.111111,0.783440,6.180483,9.458086,NEGATIVE
3,"Фонд ""Федерация"" отсудил у ""Коммерсанта"" три т...",**Фонд «Федерация» отсудил у «Коммерсанта» три...,1,YandexGPT,\n\nБлаготворительный фонд «Федерация» выиграл...,благотворительный фонд федерация выиграл дело ...,иск фонда федерация к газете коммерсантъ по по...,"Иск фонда ""Федерация"" к газете ""Коммерсантъ"" п...",0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.719282,9.139601,17.563833,NEUTRAL
4,Российские банкиры считают невозможным снижени...,**Российские банкиры считают невозможным сниже...,1,YandexGPT,\n\nВ условиях экономической нестабильности и ...,в условиях экономической нестабильности и рост...,снижение ставок по ипотечным кредитам невозмож...,Снижение ставок по ипотечным кредитам невозмож...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.815478,7.054786,8.867599,NEGATIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24399,В Арктике нашли тропические леса,Новостная статья: В Арктике нашли тропические ...,1,RuGPT3_small pre-trained,", которые могут быть долгожителями. Ученые пр...",которые могут быть долгожителями ученые проана...,британские ученые в ходе экспедиции на норвежс...,Британские ученые в ходе экспедиции на норвежс...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.570741,18.511623,10.586896,NEUTRAL
24400,Доллар упал ниже 70 рублей,Новостная статья: Доллар упал ниже 70 рублей в...,1,RuGPT3_small pre-trained,впервые с начала года. Об этом свидетельству...,впервые с начала года об этом свидетельствуют ...,курс доллара на московской бирже вечером в сре...,Курс доллара на Московской бирже вечером в сре...,0.103448,0.078947,0.150000,0.0,0.0,0.0,0.068966,0.0526315789473684,0.100000,0.706034,13.230332,6.446386,NEGATIVE
24401,Подсчитаны доходы Роналду от Instagram,Новостная статья: Подсчитаны доходы Роналду от...,1,RuGPT3_small pre-trained,-аккаунта Роналдо. Роналдиньо является самым ...,аккаунта роналдо роналдиньо является самым выс...,нападающий мадридского реала криштиану роналду...,Нападающий мадридского «Реала» Криштиану Ронал...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.675326,21.669037,9.384760,NEGATIVE
24402,Ученые нашли у кофе способность продлевать жизнь,Новостная статья: Ученые нашли у кофе способно...,1,RuGPT3_small pre-trained,"бактерий, которые производят кофеин. Исследо...",бактерий которые производят кофеин исследовани...,дветри чашки кофе в день снижают риск смерти о...,Две-три чашки кофе в день сн

In [4]:
from transformers import pipeline

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_name = 'blanchefort/rubert-base-cased-sentiment'
sentiment_pipeline = pipeline("sentiment-analysis", model=model_name, max_length = 512, truncation=True, device=device)

batch_size = 64

def get_batch_sentiments(text_list):
    text_list = [text if isinstance(text, str) else "" for text in text_list]  # Убедиться, что все элементы - строки
    results = sentiment_pipeline(text_list)
    return [result['label'] for result in results]
    
    
generated_sentiments = []
for i in tqdm(range(0, len(df['generated_text_clean']), batch_size)):
    batch_texts = df['generated_text_clean'][i:i+batch_size].tolist()  # Преобразование в список
    generated_sentiments.extend(get_batch_sentiments(batch_texts))

df['generated_sentiment'] = generated_sentiments    

clean_sentiments = []
for i in tqdm(range(0, len(df['clean_text']), batch_size)):
    batch_texts = df['clean_text'][i:i+batch_size].tolist()  # Преобразование в список
    clean_sentiments.extend(get_batch_sentiments(batch_texts))

df['clean_sentiment'] = clean_sentiments


def compare_sentiments(row):
    if row['generated_sentiment'] == row['clean_sentiment']:
        return 'same'
    else:
        return 'different'

df['sentiment_comparison'] = df.apply(compare_sentiments, axis=1)

print(df[['generated_text_clean', 'clean_text', 'generated_sentiment', 'clean_sentiment', 'sentiment_comparison']])

df.to_csv('/home/jupyter/datasphere/project/df_with_sentiment_comparison.csv', index=False)


/home/jupyter/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
100%|██████████| 381/381 [06:46<00:00,  1.07s/it]


                                    generated_text_clean  ... sentiment_comparison
0      правоохранительные органы испании арестовали г...  ...                 same
1      сегодня в жилом доме расположенном в московско...  ...            different
2      генеральная прокуратура россии провела проверк...  ...                 same
3      благотворительный фонд федерация выиграл дело ...  ...            different
4      в условиях экономической нестабильности и рост...  ...                 same
...                                                  ...  ...                  ...
24399  которые могут быть долгожителями ученые проана...  ...                 same
24400  впервые с начала года об этом свидетельствуют ...  ...            different
24401  аккаунта роналдо роналдиньо является самым выс...  ...            different
24402  бактерий которые производят кофеин исследовани...  ...            different
24403  оригинальное название trinity blade the night ...  ...            different

[24

In [2]:
sent = pd.read_csv('/home/jupyter/datasphere/project/df_with_sentiment_comparison.csv')
sent

,Unnamed: 0,title,generated_text,generated,model,generated_text_without_title,generated_text_clean,clean_text,text,cosine_similarity_calculation_,rouge,rouge1_f1,rouge1_precision,rouge1_recall,rouge2_f1,rouge2_precision,rouge2_recall,rougeL_f1,rougeL_precision,rougeL_recall,sts_score,perplexity,generated_sentiment,clean_sentiment,sentiment_comparison
0,0,В Генпрокуратуре заявили о наличии обманутых д...,**В Генпрокуратуре заявили о наличии обманутых...,1,YandexGPT,\n\nГенеральная прокуратура России провела про...,генеральная прокуратура россии провела проверк...,проблемы с обманутыми дольщиками существуют в ...,Проблемы с обманутыми дольщиками существуют в ...,-0.544019,"{'rouge1': Score(precision=1.0, recall=0.11111...",0.200000,1.000000,0.111111,0.0,0.0,0.0,0.200000,1.000000,0.111111,0.783440,7.597710,NEGATIVE,NEGATIVE,same
1,1,Северная Корея завершила создание национальных...,**Северная Корея завершила создание национальн...,1,YandexGPT,****\n\nСеверная Корея объявила о завершении с...,северная корея объявила о завершении создания ...,северная корея в 2017 году завершила создание ...,Северная Корея в 2017 году завершила создание ...,-0.593805,"{'rouge1': Score(precision=0.0, recall=0.0, fm...",0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.886655,9.210662,NEUTRAL,NEGATIVE,different
2,2,AliEхpress начал торговать холодильниками,**AliEхpress начал торговать холодильниками**\...,1,YandexGPT,****\n\nПопулярный маркетплейс AliEхpress расш...,популярный маркетплейс alieхpress расширил асс...,китайский интернетмагазин tmall работающий в р...,"Китайский интернет-магазин Tmall, работающий в...",-0.575892,"{'rouge1': Score(precision=0.0, recall=0.0, fm...",0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.625894,12.646201,POSITIVE,NEGATIVE,different
3,3,Шварценеггер пообещал все исправить в новом «Т...,**Шварценеггер пообещал все исправить в новом ...,1,YandexGPT,\n\nГолливудский актёр и экс-губернатор Калифо...,голливудский актёр и эксгубернатор калифорнии ...,голливудский актер арнольд шварценеггер расска...,Голливудский актер Арнольд Шварценеггер расска...,-0.599017,"{'rouge1': Score(precision=1.0, recall=0.16666...",0.285714,1.000000,0.166667,0.0,0.0,0.0,0.285714,1.000000,0.166667,0.751297,7.709706,NEUTRAL,NEUTRAL,same
4,4,Жертвами ДТП во Франции стали 12 человек,Вчера во Франции произошло крупное ДТП с траги...,1,YandexGPT,Вчера во Франции произошло крупное ДТП с траги...,вчера во франции произошло крупное дтп с траги...,во франции в результате лобового столкновения ...,Во Франции в результате лобового столкновения ...,-0.563112,"{'rouge1': Score(precision=1.0, recall=0.08333...",0.153846,1.000000,0.083333,0.0,0.0,0.0,0.153846,1.000000,0.083333,0.763908,9.015279,POSITIVE,NEGATIVE,different
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5993,5997,Глава Международного паралимпийского комитета ...,Новостная статья: Глава Международного паралим...,1,RuGPT3_small,"в атаке на арены Сочи\n\nВ связи с тем, что ...",в атаке на арены сочи в связи с тем что в сент...,глава международного паралимпийского комитета ...,Глава Международного паралимпийского комитета ...,-0.537662,{'rouge1': Score(precision=0.04166666666666666...,0.058824,0.041667,0.100000,0.0,0.0,0.0,0.058824,0.041667,0.100000,0.602966,15.389762,NEUTRAL,NEGATIVE,different
5994,5998,Девушка избила грабителя и заставила молить о ...,Новостная статья: Девушка избила грабителя и з...,1,RuGPT3_small,\n\nТело девушки было найдено в городе Геттис...,тело девушки было найдено в городе геттисберг ...,жительница бразильского города манаус штат ама...,"Жительница бразильского города Манаус, штат Ам...",-0.547182,"{'rouge1': Score(precision=0.0, recall=0.0, fm...",0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.512841,10.950739,NEGATIVE,NEGATIVE,same
5995,5999,Ветер сорвал крышу жилого дома в Ставрополе,Новостная статья: Ветер сорвал крышу жилого до...,1,RuGPT3_small,\n\n\nДе

In [5]:
df.groupby(by=['model', 'generated_sentiment', 'sentiment_comparison'])['clean_sentiment'].agg(['count'])

count
model                    generated_sentiment sentiment_comparison       
GigaChat                 NEGATIVE            different               339
                                             same                    562
                         NEUTRAL             different               652
                                             same                   1297
                         POSITIVE            different              1290
                                             same                    206
RuGPT3_small             NEGATIVE            different               719
                                             same                    962
                         NEUTRAL             different               418
                                             same                    547
                         POSITIVE            different               300
                                             same                     33
RuGPT3_small pre-trained NEGATIVE            different               437
                                             same                    521
                         NEUTRAL             different               120
                                             same                    170
                         POSITIVE            different                42
                                             same                      7
YandexGPT                NEGATIVE            different               492
                                             same                   1161
                         NEUTRAL             different              1831
                                             same                   2451
                         POSITIVE            different              1480
                                             same                    288
YandexGPT 3 lite         NEGATIVE            different               171
                                             same                    270
                         NEUTRAL             different               303
                                             same                    522
                         POSITIVE            different               718
                                             same                    122
YandexGPT lite           NEGATIVE            different               616
                                             same                   1383
                         NEUTRAL             different              1263
                                             same                   1722
                         POSITIVE            different               789
                                             same                    167

In [24]:
sent.loc[10].sentiment_comparison

'different'

In [4]:
sent.loc[10].clean_text

'жительница города бандаберг австралия опубликовала на своей странице в соцсети facebook снимок на котором позировала обнаженной фотография распространилась в интернете и принесла женщине популярность как пишет daily mail 31летняя кристал гордон krystal gordon приняла участие в откровенной фотосессии чтобы похвастаться своим телом после рождения первого ребенка британка поправилась сев на диету она добилась значительных результатов и решила запечатлеть их на фото в бикини однако многие пользователь сети негативно отреагировали на результаты гордон по словам британки она получила множество сообщений от ненавистников и пожаловалась на травлю в сети она не стала останавливаться на достигнутом и в следующий раз снялась без одежды кристал собирается и дальше вдохновлять полных женщин своим примером призывая любить свое тело в апреле стало известно что старшая сестра американской телезвезды ким кардашьян 38летняя кортни снялась обнаженной и опубликовала фотографию на своей странице в соцсети

In [23]:
sent.loc[10].title

'Обнаженный снимок принес полной австралийке популярность'

In [5]:
sent.loc[10].generated_text_clean

'австралийка по имени эшли грэм известная как модель plussize стала ещё популярнее после того как опубликовала в социальных сетях снимок на котором она полностью обнажена эшли грэм  известная модель которая представляет одежду больших размеров и борется за права женщин с пышными формами она часто публикует в социальных сетях свои фотографии в модных образах и купальниках недавно модель поделилась с подписчиками новым снимком который вызвал большой интерес и обсуждение в интернете на фотографии эшли грэм полностью обнажена но её тело прикрыто лишь прозрачной занавеской снимок был сделан в ванной комнате и демонстрирует уверенность модели в себе и своей внешности многие пользователи социальных сетей выразили восхищение смелостью и уверенностью эшли грэм некоторые отметили что она является примером для женщин с пышными формами и помогает им принять свою внешность снимок также вызвал дискуссию о стандартах красоты и о том как общество воспринимает женщин с разными фигурами эшли грэм продол